# Data Cleaning and Selection Patientendaten

## Imports and Read

In [3]:
import pandas as pd
import datetime
import numpy as np
import re

In [4]:
print("Start Clean and Preprocessing data patient")

Start Clean and Preprocessing data patient


In [5]:
df = pd.read_excel(r'../raw_data/Hypophysenpatienten.xlsx',sheet_name='no duplicate PID')

In [6]:
df.columns

Index(['%ID', 'Fall Nr.', 'Datum/Zeit', 'Modalität', 'Exam Code', 'Exam Name',
       'Abteilung', 'Arbeitsplatz.Kürzel', 'Aufnahmeart', 'PID', 'Grösse',
       'Ausfälle prä', 'Prolaktin', 'IGF1', 'Cortisol', 'fT4',
       'weiteres Labor', 'Qualität', 'ED', 'OP Datum', 'Ausfälle post',
       'Diagnose', 'Kategorie', 'Patient Alter', 'Zuweiser',
       'AnforderungDatum', 'ÜberweiserIntern.Bereich',
       'ÜberweiserIntern.Klinik'],
      dtype='object')

## Basic Cleaning, Column Selection, Anomaly Correction and Format definition

In [7]:
# define needed columns
column_list = ['Fall Nr.','PID',"Datum/Zeit","Arbeitsplatz.Kürzel",'Grösse',
       'Ausfälle prä', 'Qualität', 'ED','OP Datum', 'Ausfälle post',
       'Diagnose', 'Kategorie', 'Patient Alter',
       'Prolaktin',"IGF1", 'Cortisol','fT4','weiteres Labor']


### Data Type Definition


In [8]:
#TODO: check Tristan
# not parseable correct values corrected
df.loc[3,'ED'] = datetime.datetime(2006,1,1,0,0,0,0)
df.loc[12,'ED'] = datetime.datetime(2008,1,1,0,0,0,0)

#TODO: check Tristan
# correct a value which is not datetime parseable
df.loc[df['OP Datum'] == '2006, 2009', 'OP Datum'] = datetime.datetime(2006,1,1,0,0,0,0)

In [9]:
# make datetime values
df["Datum/Zeit"] = pd.to_datetime(df["Datum/Zeit"])
df["ED"] = pd.to_datetime(df["ED"])
df["OP Datum"] = pd.to_datetime(df["OP Datum"])

In [10]:
# TODO: anomaly? check tristan
# rows where Entry Date is after Operationdate?
df[df['OP Datum'] < df['ED']][['ED','OP Datum']]

,ED,OP Datum
20,2023-09-02,2009-02-18


### Impute NAN Lab Data

In [11]:
# TODO: set impute value
df.loc[df["Prolaktin"].isna(),'Prolaktin'] = '0'
df.loc[df["IGF1"].isna(),'IGF1'] = '0'
df.loc[df["Cortisol"].isna(),'Cortisol'] = '0'
df.loc[df["fT4"].isna(),'fT4'] = '0'


### Unit Conversion Lab Data

In [12]:
df["Prolaktin"].unique()

array(['0', '173mU/l', '743mU/l', '687mU/l', '7.8 ug/l'], dtype=object)

In [13]:
# get indices which need to be converted
indices_to_divide = df.loc[df["Prolaktin"].str.contains('ug/l'),'Prolaktin'].index 
# remove units and strings
df['Prolaktin'] = df['Prolaktin'].str.rstrip(r'mU/l')
df['Prolaktin'] = df['Prolaktin'].str.rstrip(r'ug/l')
df['Prolaktin'] = df['Prolaktin'].astype(float)
# ug/l -> mU/l (ug/l * 21.2)
df.loc[indices_to_divide,'Prolaktin'] = df.loc[indices_to_divide,'Prolaktin'] * 21.2


In [14]:
df["IGF1"].unique()

array(['0', '6.3nmol/l', '20.2nmol/l', '75.4ng/ml', '208 ng/ml'],
      dtype=object)

In [15]:
# get indices which need to be converted
indices_to_divide = df.loc[df["IGF1"].str.contains('ng/ml'),'IGF1'].index 
# remove units and strings
df['IGF1'] = df['IGF1'].str.rstrip(r'nmol/l')
df['IGF1'] = df['IGF1'].str.rstrip(r'ng/ml')
df['IGF1'] = df['IGF1'].astype(float)
# ng/ml -> nmol/l (ng/ml / 7.65)
df.loc[indices_to_divide,'IGF1'] = df.loc[indices_to_divide,'IGF1'] / 7.65


In [16]:
df["Cortisol"].unique()

array(['0', 329, 271, 110, '311 nmol/l'], dtype=object)

In [17]:
# # get indices which need to be converted
# indices_to_divide = df.loc[df["Cortisol"].str.contains('ng/ml'),'Cortisol'].index 
# # remove units and strings
# df['Cortisol'] = df['Cortisol'].str.rstrip(r'nmol/l')
# df['Cortisol'] = df['IGF1'].str.rstrip(r'ng/ml')
# df['Cortisol'] = df['Cortisol'].astype(float)
# # ng/ml -> nmol/l (ng/ml *  27.59)
# df.loc[indices_to_divide,'Cortisol'] = df.loc[indices_to_divide,'Cortisol'] / 7.65


In [18]:
df["fT4"].unique()

array(['0', 10.1, 8.4, 7.3, '14.6 pmol/l'], dtype=object)

In [19]:
# # get indices which need to be converted
# indices_to_divide = df.loc[df["fT4"].str.contains('ng/ml'),'fT4'].index 
# # remove units and strings
# df['fT4'] = df['fT4'].str.rstrip(r'nmol/l')
# df['fT4'] = df['fT4'].str.rstrip(r'ng/ml')
# df['fT4'] = df['fT4'].astype(float)
# # ng/ml -> nmol/l (ng/ml / 7.65)
# df.loc[indices_to_divide,'fT4'] = df.loc[indices_to_divide,'fT4'] / 7.65


In [20]:
# Patient ID Duplicate Check
assert len(df[df["PID"].duplicated()]) == 0

# Case Nr Duplicate Check
assert len(df[df["Fall Nr."].duplicated()]) == 0

In [21]:
# select and rename columns
df = df[column_list]
df= df.rename(columns={"Fall Nr.": "Case_ID","PID": "Patient_ID",
                       "Datum/Zeit": "Date_MRI","ED": "Entry_date", "OP Datum": "Operation_date",
                       "Arbeitsplatz.Kürzel":"ID_MRI_Machine","Grösse": "Adenoma_size","Qualität": "Label_Quality",
                       "Patient Alter":"Patient_age","Kategorie":"Category","Diagnose":"Diagnosis",
                       "Prolaktin":"Prolactin","weiteres Labor":"Lab_additional"})

In [22]:
# set category data type in pandas, check datatypes
df['ID_MRI_Machine'] = df['ID_MRI_Machine'].astype('category')
df['Adenoma_size'] = df['Adenoma_size'].astype('category')
df['Label_Quality'] = df['Label_Quality'].astype('category')
df['Diagnosis'] = df['Diagnosis'].astype('category')
df['Category'] = df['Category'].astype('category')
df.dtypes

Case_ID                    int64
Patient_ID                 int64
Date_MRI          datetime64[ns]
ID_MRI_Machine          category
Adenoma_size            category
Ausfälle prä              object
Label_Quality           category
Entry_date        datetime64[ns]
Operation_date    datetime64[ns]
Ausfälle post             object
Diagnosis               category
Category                category
Patient_age                int64
Prolactin                float64
IGF1                     float64
Cortisol                  object
fT4                       object
Lab_additional            object
dtype: object

In [23]:
# replace and correct wrong namings from labelers
df["Ausfälle prä"]= df["Ausfälle prä"].str.replace("intak","intakt")
df["Ausfälle prä"]= df["Ausfälle prä"].str.replace("goando","gonado")
df["Ausfälle post"]= df["Ausfälle post"].str.replace("goando","gonado")
df["Ausfälle post"]= df["Ausfälle post"].str.replace("adh","ADH")

## One Hot Encode Categorical Values

To use and analyse the categorical data we need to one-hot encode them. This is done by splitting the comma separated strings into single strings and then create a one-hot-encoded column of each individual value. This column is then added to the original dataframe.

In [24]:
# Split the 'Ausfälle prä' column into separate strings
df['Ausfälle prä'] = df['Ausfälle prä'].str.split(', ')
# Create a set to store all unique disfunctions
unique_disfunctions = set()

# Iterate over the 'Ausfälle prä' column to gather unique disfunctions
for value in df['Ausfälle prä']:
    if isinstance(value, list):
        unique_disfunctions.update(value)
    elif isinstance(value, str):
        unique_disfunctions.add(value)

# Iterate over the unique disfunctions and create one-hot encoded columns
for disfunction in unique_disfunctions:
    df["Pre_OP_hormone_"+ disfunction] = df['Ausfälle prä'].apply(lambda x: 1 if (isinstance(x, list) and disfunction in x) or (x == disfunction) else 0)
# drop the original 'Ausfälle prä' column
df = df.drop('Ausfälle prä', axis=1)

In [25]:
# Split the 'Ausfälle post' column into separate strings
df['Ausfälle post'] = df['Ausfälle post'].str.split(', ')

# Create a set to store all unique disfunctions
unique_disfunctions = set()

# Iterate over the 'Ausfälle post' column to gather unique disfunctions
for value in df['Ausfälle post']:
    if isinstance(value, list):
        unique_disfunctions.update(value)
    elif isinstance(value, str):
        unique_disfunctions.add(value)

# Iterate over the unique disfunctions and create one-hot encoded columns
for disfunction in unique_disfunctions:
    df["Post_OP_hormone_"+ disfunction] = df['Ausfälle post'].apply(lambda x: 1 if (isinstance(x, list) and disfunction in x) or (x == disfunction) else 0)

# drop the original 'Ausfälle post' column
df = df.drop('Ausfälle post', axis=1)

In [26]:
df.columns

Index(['Case_ID', 'Patient_ID', 'Date_MRI', 'ID_MRI_Machine', 'Adenoma_size',
       'Label_Quality', 'Entry_date', 'Operation_date', 'Diagnosis',
       'Category', 'Patient_age', 'Prolactin', 'IGF1', 'Cortisol', 'fT4',
       'Lab_additional', 'Pre_OP_hormone_', 'Pre_OP_hormone_cortico',
       'Pre_OP_hormone_thyreo', 'Pre_OP_hormone_hyperprolaktin',
       'Pre_OP_hormone_coritco', 'Pre_OP_hormone_hypothyreo',
       'Pre_OP_hormone_hypogonado',
       'Pre_OP_hormone_hyperprolakin stressinduziert',
       'Pre_OP_hormone_Kompression', 'Pre_OP_hormone_inaktiv',
       'Pre_OP_hormone_somato', 'Pre_OP_hormone_gondao',
       'Pre_OP_hormone_SIADH', 'Pre_OP_hormone_intakt',
       'Pre_OP_hormone_hyperprolaktin stressbedingt',
       'Pre_OP_hormone_morbus cushing', 'Pre_OP_hormone_gonado',
       'Pre_OP_hormone_keine', 'Post_OP_hormone_cortico',
       'Post_OP_hormone_thyreo', 'Post_OP_hormone_hyperprolaktin',
       'Post_OP_hormone_Proliferationsmarker nicht nachgewiesen',
     

In [27]:
df= df[['Case_ID', 'Patient_ID', 'Date_MRI', 'ID_MRI_Machine','Entry_date', 'Operation_date', 'Adenoma_size',
         'Diagnosis',
       'Category', 'Patient_age', 'Prolactin',"IGF1", 'Cortisol','fT4','Lab_additional',
       'Pre_OP_hormone_cortico', 'Pre_OP_hormone_gonado',
       'Pre_OP_hormone_somato', 'Pre_OP_hormone_thyreo',
       'Pre_OP_hormone_hyperprolaktin', 'Pre_OP_hormone_keine',
       'Pre_OP_hormone_intakt', 'Post_OP_hormone_cortico',
       'Post_OP_hormone_gonado', 'Post_OP_hormone_somato',
       'Post_OP_hormone_ADH', 'Post_OP_hormone_thyreo',
       'Post_OP_hormone_hyperprolaktin', 'Post_OP_hormone_keine',
       'Post_OP_hormone_intakt', 'Label_Quality',]]

In [28]:
df.to_csv(r'../raw_data/label_data.csv',index=False)

In [29]:
print("End Clean and Preprocessing patient data")

End Clean and Preprocessing patient data


# Data Cleaning and Selection Labor data

In [30]:
print("Start Clean and Preprocessing labor data")

Start Clean and Preprocessing labor data


## Read

In [31]:
labor_data_raw = pd.read_excel("../raw_data/labor_data.xlsx")
labor_data = pd.DataFrame({})


In [32]:
# change float with , to float with .
labor_data[labor_data_raw.columns[0]] = [re.sub('"([^",]+),([^"]+)"',r"\1.\2", re.sub('"([^",]+),([^",]+),([^"]+)"',r"\1.\2.\3", string[0])) for string in labor_data_raw.values]

In [33]:
labor_data= labor_data.iloc[:,0].str.split(",",expand=True)
labor_data.columns = labor_data_raw.columns[0].split(",")

In [34]:
labor_data[labor_data=="NULL"] = np.nan

In [35]:
ids = {'Ã¼': 'ü', 'Ã¤': 'ä', "Ã„":"Ä"}

for column in labor_data.columns:
    for old, new in ids.items():
        labor_data[column] = labor_data[column].str.replace(old, new, regex=False)
        
labor_data["Normwert"] = labor_data["Normwert"].str.lower().str.replace(" ", "").replace("",np.nan)
labor_data["Warnung"] = labor_data["Warnung"].replace('   ',np.nan)
labor_data["Datum_Resultat"] = pd.to_datetime(labor_data["Datum_Resultat"],dayfirst=True)
labor_data["PATIENT_NR"] = labor_data["PATIENT_NR"].astype(int)

In [56]:
lst = []
for Nr in labor_data["FALL_NR"]:
    try:
        lst.append(int(Nr))
    except:
        lst.append(np.nan)

In [43]:
# check for transformation in unit
for value in labor_data["Analyse-ID"].unique():
    if len(labor_data[labor_data["Analyse-ID"] == value].loc[:,["Analyse-ID","Einheit"]].drop_duplicates()) !=1:
        raise ValueError

In [44]:
values = []
for value in labor_data["Resultat"]:
    try:
        float(value)
    except:
        values.append(value)

In [45]:
np.unique(np.array(values))

array(['1Â\xa0003', '1Â\xa0004', '1Â\xa0005', '1Â\xa0006', '1Â\xa0007',
       '1Â\xa0009', '1Â\xa0011', '1Â\xa0016', '1Â\xa0017', '1Â\xa0018',
       '1Â\xa0019', '1Â\xa0021', '1Â\xa0027', '1Â\xa0028', '1Â\xa0029',
       '1Â\xa0031', '1Â\xa0034', '1Â\xa0035', '1Â\xa0038', '1Â\xa0041',
       '1Â\xa0043', '1Â\xa0045', '1Â\xa0047', '1Â\xa0049', '1Â\xa0050',
       '1Â\xa0051', '1Â\xa0052', '1Â\xa0053', '1Â\xa0055', '1Â\xa0058',
       '1Â\xa0059', '1Â\xa0061', '1Â\xa0066', '1Â\xa0070', '1Â\xa0071',
       '1Â\xa0075', '1Â\xa0076', '1Â\xa0079', '1Â\xa0082', '1Â\xa0083',
       '1Â\xa0084', '1Â\xa0085', '1Â\xa0087', '1Â\xa0088', '1Â\xa0089',
       '1Â\xa0093', '1Â\xa0094', '1Â\xa0096', '1Â\xa0097', '1Â\xa0098',
       '1Â\xa0100', '1Â\xa0104', '1Â\xa0105', '1Â\xa0113', '1Â\xa0116',
       '1Â\xa0118', '1Â\xa0119', '1Â\xa0121', '1Â\xa0125', '1Â\xa0126',
       '1Â\xa0133', '1Â\xa0136', '1Â\xa0145', '1Â\xa0146', '1Â\xa0147',
       '1Â\xa0149', '1Â\xa0152', '1Â\xa0156', '1Â\xa0159', '1Â\x

In [46]:
labor_data[labor_data["Resultat"] == "1Â\xa0896"]

,FALL_NR,PATIENT_NR,Analyse,Analyse-ID,Auftraggeber,Datum_Resultat,Resultat,Einheit,Normwert,Fallart,Warnung
17541,10308286,261047,Cortisol-30' Synacthen-Test,COR30,SPITAL ZOFINGEN,2021-03-11,1Â 896,nmol/l,>550,no,NaN
